# Passing input arguments to your QiskitFunction

In this document, we will learn how to pass arguments to our function.

Let's create another file with our function [./source_files/function_with_arguments.py](./source_files/function_with_arguments.py). 

Instead of having the circuit defined inside the function (like we did in the first example), we will pass it as an argument. To make this case more realistic, we will also pass a backend argument for the transpilation and the Sampler. This use case is for transpiling a circuit in the cloud. We will also save the results, so we can access them later by calling [save_result](https://qiskit.github.io/qiskit-serverless/stubs/qiskit_serverless.core.save_result.html#qiskit_serverless.core.save_result).

Here is the function:

```python
from qiskit import QuantumCircuit
from qiskit.providers import BackendV2
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_serverless import get_arguments, save_result
from qiskit_ibm_runtime import SamplerV2 as Sampler


# get all arguments passed to this function
arguments = get_arguments()

# get specific argument that we are interested in
circuit = arguments.get("circuit")
backend = arguments.get("backend")

# verifying arguments types
assert isinstance(circuit, QuantumCircuit)
assert isinstance(backend, BackendV2)

# matching our run to the backend argument
sampler = Sampler(backend)
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
isa_circuit = pm.run(circuit)

# running the circuit
quasi_dists = sampler.run([isa_circuit]).result()[0].data.meas.get_counts()

print(f"Quasi distribution: {quasi_dists}")

# saving results of the execution
save_result({
    "quasi_dists": quasi_dists
})
```

As you can see, the circuit construction is not inside the function anymore. Instead, we parse the arguments by calling the [get_arguments](https://qiskit.github.io/qiskit-serverless/stubs/qiskit_serverless.serializers.get_arguments.html#qiskit_serverless.serializers.get_arguments) function.

First, we will create circuit that we want to pass as an argument to the function.

In [1]:
from qiskit import QuantumCircuit

circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()
circuit.draw()

┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1

Then, we will create our `BackendV2` object.

In [ ]:
from qiskit_aer import AerSimulator

backend = AerSimulator()

Now let's create and configure our client

> &#x26A0; This provider is set up with default credentials to a test cluster intended to run on your machine. For information on setting up infrastructure on your local machine, check out the guide on [local infrastructure setup](https://qiskit.github.io/qiskit-serverless/deployment/local.html).

In [ ]:
from qiskit_serverless import ServerlessClient
import os

client = ServerlessClient(
    token=os.environ.get("GATEWAY_TOKEN", "awesome_token"),
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
    # If you are using the kubernetes approach the URL must be http://localhost
)
client

<gateway-client>

In [3]:
from qiskit_serverless import QiskitFunction

function = QiskitFunction(
    title="function-with-arguments",
    entrypoint="function_with_arguments.py",
    working_dir="./source_files/",
)

client.upload(function)

QiskitFunction(function-with-arguments)

Run the function using the `run` method and by passing arguments.

In [4]:
my_function = client.get("function-with-arguments")
my_function

QiskitFunction(function-with-arguments)

In [5]:
job = my_function.run(circuit=circuit, backend=backend)
job

<Job | a2cfbcdc-f503-4be3-9bcf-8914ee110bb2>

Retrieve the results from the client

In [6]:
job.result()

{'quasi_dists': {'11': 524, '00': 500}}